In [1]:
import os
import numpy as np
import scanpy as sc
import anndata as ad
import pandas as pd
import re
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
plt.rcParams['figure.figsize']=(5, 5)
sc.settings.verbosity = 3
sc.logging.print_header()
os.chdir('./../')


%load_ext autoreload
%autoreload 2 

scanpy==1.7.1 anndata==0.7.5 umap==0.4.6 numpy==1.19.4 scipy==1.5.4 pandas==1.1.4 scikit-learn==0.23.2 statsmodels==0.12.1 python-igraph==0.8.3 leidenalg==0.8.3


In [2]:
sc.set_figure_params(dpi=100)

## loading the raw data

In [3]:
import os

In [4]:
os.getcwd()

'/home/mohammad/Desktop/test_AdvAE'

In [33]:
adata = sc.read("./datasets/pachter_raw.h5ad")

/home/mohammad/anaconda3/envs/scvi-env/lib/python3.7/site-packages/anndata/compat/__init__.py:182: FutureWarning: Moving element from .uns['neighbors']['distances'] to .obsp['distances'].

This is where adjacency matrices should go now.
  FutureWarning,
/home/mohammad/anaconda3/envs/scvi-env/lib/python3.7/site-packages/anndata/compat/__init__.py:182: FutureWarning: Moving element from .uns['neighbors']['connectivities'] to .obsp['connectivities'].

This is where adjacency matrices should go now.
  FutureWarning,


In [10]:
adata_anna = sc.read("./datasets/pachter.h5ad")

undoing some reindexing done by scanpy

In [25]:
new_index = []
for obs in adata_anna.obs.index:
    split = obs.split("-")
    new_index.append(split[0]+"-"+split[1])

In [31]:
adata_anna.obs.index = new_index

In [37]:
len(set(adata_anna.obs.index).intersection(adata.obs.index)) == len(adata_anna)

True

In [38]:
adata = adata[new_index,:]

In [46]:
raw = adata.raw.to_adata()

the original data is already normalized 

In [50]:
raw.X.max()

8.53352

In [56]:
adata_new = sc.AnnData(X=raw.X, obs=adata_anna.obs, uns=adata_anna.uns, var=raw.var)

In [57]:
adata_new.var_names

Index(['Xkr4', 'Gm1992', 'Rp1', 'Mrpl15', 'Lypla1', 'Gm37988', 'Tcea1',
       'Rgs20', 'Atp6v1h', 'Rb1cc1',
       ...
       'Vamp7', 'Spry3', 'Tmlhe', 'AC132444.1', 'Csprs', 'AC125149.3',
       'AC168977.1', 'PISD', 'DHRSX', 'CAAA01147332.1'],
      dtype='object', name='index', length=16950)

In [58]:
adata_new.obs

,BMP,EGF,PlatePos,RA,ScripDec,batch,cell_type,cell_types,comb,condition,control,dose_val,louvain,n_counts,n_countslog,split,cov_drug_dose_name
CTGATAGCAATGGACG-2,0.8,20.0,8,0,0,0,unknown,11,"0R,0.8B,20.0E,0S",BMP+EGF,0,0.04+1.0,11,2054.0,3.312600,train,unknown_BMP+EGF_0.04+1.0
CATCCACGTAGTGAAT-1,0.8,20.0,18,5,0,0,unknown,10,"5R,0.8B,20.0E,0S",BMP+EGF+RA,0,0.04+1.0+1.0,10,3832.0,3.583426,train,unknown_BMP+EGF+RA_0.04+1.0+1.0
AGTAGTCAGTGGAGTC-2,0.0,0.8,59,0,0,0,unknown,14,"0R,0.0B,0.8E,0S",EGF,1,0.04,14,6045.0,3.781396,train,unknown_EGF_0.04
TGGGCGTAGAATTCCC-1,0.8,20.0,18,5,0,0,unknown,6,"5R,0.8B,20.0E,0S",BMP+EGF+RA,0,0.04+1.0+1.0,6,7974.0,3.901676,train,unknown_BMP+EGF+RA_0.04+1.0+1.0
TGGCCAGCACATGACT-2,4.0,0.8,63,5,0,0,unknown,13,"5R,4.0B,0.8E,0S",BMP+EGF+RA,0,0.2+0.04+1.0,13,2576.0,3.410946,train,unknown_BMP+EGF+RA_0.2+0.04+1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TGTATTCTCAGGTTCA-4,4.0,20.0,16,1,0,2,unknown,5,"1R,4.0B,20.0E,0S",BMP+EGF+RA,0,0.2+1.0+0.2,5,1730.0,3.238046,ood,unknown_BMP+EGF+RA_0.2+1.0+0.2
TTAGTTCCACAGATTC-4,4.0,20.0,16,1,0,2,unknown,5,"1R,4.0B,20.0E,0S",BMP+EGF+RA,0,0.2+1.0+0.2,5,1287.0,3.109579,ood,unknown_BMP+EGF+RA_0.2+1.0+0.2
TTCTCCTCACAGATTC-4,4.0,4.0,28,0,1,2,unknown,5,"0R,4.0B,4.0E,1S",BMP+EGF+ScripDec,0,0.2+0.2+0.2,5,5099.0,3.707485,ood,unknown_BMP+EGF+ScripDec_0.2+0.2+0.2
TTTGCGCCAAGACACG-4,4.0,4.0,28,0,1,2,unknown,3,"0R,4.0B,4.0E,1S",BMP+EGF+ScripDec,0,0.2+0.2+0.2,3,6342.0,3.802226,ood,unknown_BMP+EGF+ScripDec_0.2+0.2+0.2


# preprocessing 

Keep the all genes in adata .raw

In [64]:
adata_new.raw = adata_new

Normalization and HVG selection

In [65]:
sc.pp.highly_variable_genes(adata_new,n_top_genes=5000, subset=True)

If you pass `n_top_genes`, all cutoffs are ignored.
extracting highly variable genes
    finished (0:00:00)
--> added
    'highly_variable', boolean vector (adata.var)
    'means', float vector (adata.var)
    'dispersions', float vector (adata.var)
    'dispersions_norm', float vector (adata.var)


In [68]:
adata_new.obs

,BMP,EGF,PlatePos,RA,ScripDec,batch,cell_type,cell_types,comb,condition,control,dose_val,louvain,n_counts,n_countslog,split,cov_drug_dose_name
CTGATAGCAATGGACG-2,0.8,20.0,8,0,0,0,unknown,11,"0R,0.8B,20.0E,0S",BMP+EGF,0,0.04+1.0,11,2054.0,3.312600,train,unknown_BMP+EGF_0.04+1.0
CATCCACGTAGTGAAT-1,0.8,20.0,18,5,0,0,unknown,10,"5R,0.8B,20.0E,0S",BMP+EGF+RA,0,0.04+1.0+1.0,10,3832.0,3.583426,train,unknown_BMP+EGF+RA_0.04+1.0+1.0
AGTAGTCAGTGGAGTC-2,0.0,0.8,59,0,0,0,unknown,14,"0R,0.0B,0.8E,0S",EGF,1,0.04,14,6045.0,3.781396,train,unknown_EGF_0.04
TGGGCGTAGAATTCCC-1,0.8,20.0,18,5,0,0,unknown,6,"5R,0.8B,20.0E,0S",BMP+EGF+RA,0,0.04+1.0+1.0,6,7974.0,3.901676,train,unknown_BMP+EGF+RA_0.04+1.0+1.0
TGGCCAGCACATGACT-2,4.0,0.8,63,5,0,0,unknown,13,"5R,4.0B,0.8E,0S",BMP+EGF+RA,0,0.2+0.04+1.0,13,2576.0,3.410946,train,unknown_BMP+EGF+RA_0.2+0.04+1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TGTATTCTCAGGTTCA-4,4.0,20.0,16,1,0,2,unknown,5,"1R,4.0B,20.0E,0S",BMP+EGF+RA,0,0.2+1.0+0.2,5,1730.0,3.238046,ood,unknown_BMP+EGF+RA_0.2+1.0+0.2
TTAGTTCCACAGATTC-4,4.0,20.0,16,1,0,2,unknown,5,"1R,4.0B,20.0E,0S",BMP+EGF+RA,0,0.2+1.0+0.2,5,1287.0,3.109579,ood,unknown_BMP+EGF+RA_0.2+1.0+0.2
TTCTCCTCACAGATTC-4,4.0,4.0,28,0,1,2,unknown,5,"0R,4.0B,4.0E,1S",BMP+EGF+ScripDec,0,0.2+0.2+0.2,5,5099.0,3.707485,ood,unknown_BMP+EGF+ScripDec_0.2+0.2+0.2
TTTGCGCCAAGACACG-4,4.0,4.0,28,0,1,2,unknown,3,"0R,4.0B,4.0E,1S",BMP+EGF+ScripDec,0,0.2+0.2+0.2,3,6342.0,3.802226,ood,unknown_BMP+EGF+ScripDec_0.2+0.2+0.2


DE test

In [67]:
from compert.helper import rank_genes_groups_by_cov

In [73]:
rank_genes_groups_by_cov(adata_new, groupby='cov_drug_dose_name', covariate='cell_type', control_group='EGF_0.04', n_genes=50)

ranking genes
Trying to set attribute `.uns` of view, copying.


unknown


    finished: added to `.uns['rank_genes_groups']`
    'names', sorted np.recarray to be indexed by group ids
    'scores', sorted np.recarray to be indexed by group ids
    'logfoldchanges', sorted np.recarray to be indexed by group ids
    'pvals', sorted np.recarray to be indexed by group ids
    'pvals_adj', sorted np.recarray to be indexed by group ids (0:00:00)


saving the object

In [75]:
adata_new

AnnData object with n_obs × n_vars = 19637 × 5000
    obs: 'BMP', 'EGF', 'PlatePos', 'RA', 'ScripDec', 'batch', 'cell_type', 'cell_types', 'comb', 'condition', 'control', 'dose_val', 'louvain', 'n_counts', 'n_countslog', 'split', 'cov_drug_dose_name'
    var: 'gene_ids', 'feature_types', 'genome', 'n_counts', 'highly_variable', 'means', 'dispersions', 'dispersions_norm'
    uns: 'rank_genes_groups', 'rank_genes_groups_cov', 'hvg'

In [74]:
adata_new.obs

,BMP,EGF,PlatePos,RA,ScripDec,batch,cell_type,cell_types,comb,condition,control,dose_val,louvain,n_counts,n_countslog,split,cov_drug_dose_name
CTGATAGCAATGGACG-2,0.8,20.0,8,0,0,0,unknown,11,"0R,0.8B,20.0E,0S",BMP+EGF,0,0.04+1.0,11,2054.0,3.312600,train,unknown_BMP+EGF_0.04+1.0
CATCCACGTAGTGAAT-1,0.8,20.0,18,5,0,0,unknown,10,"5R,0.8B,20.0E,0S",BMP+EGF+RA,0,0.04+1.0+1.0,10,3832.0,3.583426,train,unknown_BMP+EGF+RA_0.04+1.0+1.0
AGTAGTCAGTGGAGTC-2,0.0,0.8,59,0,0,0,unknown,14,"0R,0.0B,0.8E,0S",EGF,1,0.04,14,6045.0,3.781396,train,unknown_EGF_0.04
TGGGCGTAGAATTCCC-1,0.8,20.0,18,5,0,0,unknown,6,"5R,0.8B,20.0E,0S",BMP+EGF+RA,0,0.04+1.0+1.0,6,7974.0,3.901676,train,unknown_BMP+EGF+RA_0.04+1.0+1.0
TGGCCAGCACATGACT-2,4.0,0.8,63,5,0,0,unknown,13,"5R,4.0B,0.8E,0S",BMP+EGF+RA,0,0.2+0.04+1.0,13,2576.0,3.410946,train,unknown_BMP+EGF+RA_0.2+0.04+1.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TGTATTCTCAGGTTCA-4,4.0,20.0,16,1,0,2,unknown,5,"1R,4.0B,20.0E,0S",BMP+EGF+RA,0,0.2+1.0+0.2,5,1730.0,3.238046,ood,unknown_BMP+EGF+RA_0.2+1.0+0.2
TTAGTTCCACAGATTC-4,4.0,20.0,16,1,0,2,unknown,5,"1R,4.0B,20.0E,0S",BMP+EGF+RA,0,0.2+1.0+0.2,5,1287.0,3.109579,ood,unknown_BMP+EGF+RA_0.2+1.0+0.2
TTCTCCTCACAGATTC-4,4.0,4.0,28,0,1,2,unknown,5,"0R,4.0B,4.0E,1S",BMP+EGF+ScripDec,0,0.2+0.2+0.2,5,5099.0,3.707485,ood,unknown_BMP+EGF+ScripDec_0.2+0.2+0.2
TTTGCGCCAAGACACG-4,4.0,4.0,28,0,1,2,unknown,3,"0R,4.0B,4.0E,1S",BMP+EGF+ScripDec,0,0.2+0.2+0.2,3,6342.0,3.802226,ood,unknown_BMP+EGF+ScripDec_0.2+0.2+0.2


In [76]:
adata_new.write("./datasets/pachter_new.h5ad")